# core 

> modifying streamlit methods to play nice with jupyter

Currently streamlit expects to find itself in a standalone python script, and will not work in a jupyter notebook. This is because streamlit is a web app, and jupyter notebooks are not.

To get around this, we can use the `get_ipython` method to check if we are in a jupyter notebook, and if so, we can use the `display` method to display the streamlit app in the notebook.

This allows us to interactively code in our jupyter notebook and see how our streamlit app will look in real time.

In [1]:
# | default_exp core

In [2]:
# | export

import logging
import time
import typing as tp

import streamlit
from fastcore.basics import in_ipython, listify, noop, patch, patch_to
from fastcore.test import test_fail, test_eq
from IPython.utils.capture import capture_output
from nbdev.showdoc import show_doc


import functools

import IPython.display

In [3]:
# | export

# module obljects that we will be importing
IN_IPYTHON = in_ipython()

In [4]:
assert IN_IPYTHON, "This module is intended to be used in a Jupyter notebook"

In [5]:
# | export
import inspect


class StreamlitPatcher:
    """class to patch streamlit functions for displaying content in jupyter notebooks"""

    def __init__(self):
        self.is_registered: bool = False
        self.registered_methods: tp.Set[str] = set()

    def jupyter(self):
        """
        Registers the current `tqdm` class with
            streamlit.
            ( write
              markdown

            )
        """
        # patch streamlit methods from MAPPING property dict
        for method_name, wrapper in self.MAPPING.items():
            self._wrap(method_name, wrapper)

        # patch stqdm

        self.is_registered = True

    @staticmethod
    def _get_streamlit_methods():
        """get all streamlit methods"""
        return [attr for attr in dir(st) if not attr.startswith("_")]

## _wrap method to patch streamlit methods

In [6]:
p = functools.partial(print, end="")

In [7]:
# | exporti
@patch_to(StreamlitPatcher, cls_method=False)
def _wrap(cls, method_name: str, wrapper: tp.Callable, forced_wrapper_name: tp.Optional[str] = '') -> None:
    """make a streamlit method jupyter friendly

    Parameters
    ----------
    method_name : str
        which method to jupyterify
    wrapper : tp.Callable
        wrapper function to use
    """
    if IN_IPYTHON:  # only patch if in jupyter
        if hasattr(wrapper, "__name__"):
            print(f"wrapping 'streamlit.{method_name}' with 'streamlit_jupyter.core.{wrapper.__name__}'")
        else:
            print(f"wrapping 'streamlit.{method_name}' with 'streamlit_jupyter.core.{wrapper}'")

        trg = getattr(streamlit, method_name)
        setattr(streamlit, method_name, wrapper(trg))
        cls.registered_methods.add(method_name)
    else:  # otherwise do nothing
        pass

In [8]:
sp = StreamlitPatcher()

assert not sp.is_registered, "StreamlitPatcher is already registered"

In [9]:
show_doc(StreamlitPatcher._wrap)

---

[source](https://github.com/ddobrinskiy/streamlit-jupyter/blob/master/streamlit_jupyter/core.py#L61){target="_blank" style="float:right; font-size:smaller"}

### StreamlitPatcher._wrap

>      StreamlitPatcher._wrap (cls, method_name:str, wrapper:Callable,
>                              forced_wrapper_name:Union[str,NoneType]='')

make a streamlit method jupyter friendly

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| cls |  |  |  |
| method_name | str |  | which method to jupyterify |
| wrapper | typing.Callable |  | wrapper function to use |
| forced_wrapper_name | typing.Union[str, NoneType] |  |  |
| **Returns** | **None** |  |  |

## Modifying streamlit

The way we will modify streamlit methods is by putting them through a decorator. This decorator will check if we are in a jupyter notebook, and if so, it will take the input and display it in the notebook. 

Else it will use the original streamlit method.

### patching streamlit.write

In [10]:
# | exporti

def _display(arg: tp.Any) -> None:
    if isinstance(arg, str):
        IPython.display.display(IPython.display.Markdown(arg))
    else:
        IPython.display.display(arg)


def _st_write(func_to_decorate):
    """Decorator to display objects passed to Streamlit in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        for arg in args:
            _display(arg)

    return wrapper

In [11]:
sp._wrap("write", _st_write)

with capture_output() as cap:
    streamlit.write("hello")
    got = cap._outputs[0]["data"]

expected = {
    "text/plain": "<IPython.core.display.Markdown object>",
    "text/markdown": "hello",
}
assert got == expected, "check that the output is correct"

streamlit.write("hello")

wrapping 'streamlit.write' with 'streamlit_jupyter.core._st_write'


hello

In [12]:
streamlit.write("This is **bold** text in markdown")

This is **bold** text in markdown

In [13]:
try:
    import pandas as pd

    df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
    streamlit.write(df)
except ImportError:
    logger.warning("Pandas not installed, skipping test")

,a,b
0,1,4
1,2,5
2,3,6


In [14]:
assert sp.registered_methods == {"write"}, "check that the method is registered"

### patching headings

- `streamlittitle`
- `streamlit.header`
- `streamlitsubheader`

In [15]:
# | exporti
def _st_heading(func_to_decorate: tp.Callable, tag: str) -> tp.Callable:
    """Decorator to display objects passed to Streamlit in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) == 1:
            body = args[0]
        elif len(args) == 2:
            body, anchor = args
        elif len(args) > 2:
            raise ValueError(
                f"Too many positional arguments: {len(args)}, {func_to_decorate.__name__} only accepts 2"
            )
        elif len(args) == 0:
            if "body" not in kwargs:
                raise ValueError(
                    f"Missing required argument: body, {func_to_decorate.__name__} requires a body"
                )
            body = kwargs["body"]

        if isinstance(body, str):
            _display(f"{tag} {body}")
        else:
            raise TypeError(
                f"Unsupported type: {type(body)}, {func_to_decorate.__name__} only accepts strings"
            )

    return wrapper

In [25]:
sp = StreamlitPatcher()
sp._wrap('title', functools.partial(_st_heading, tag="#"))
sp._wrap('header', functools.partial(_st_heading, tag="##"))
sp._wrap('subheader', functools.partial(_st_heading, tag="###"))

wrapping 'streamlit.title' with 'streamlit_jupyter.core.functools.partial(<function _st_heading at 0x12f5bcdc0>, tag='#')'
wrapping 'streamlit.header' with 'streamlit_jupyter.core.functools.partial(<function _st_heading at 0x12f5bcdc0>, tag='##')'
wrapping 'streamlit.subheader' with 'streamlit_jupyter.core.functools.partial(<function _st_heading at 0x12f5bcdc0>, tag='###')'


In [26]:
with capture_output() as cap:
    streamlit.title("foo")
    got = cap._outputs[0]["data"]['text/markdown']

test_eq(got, '# foo')

In [27]:
with capture_output() as cap:
    streamlit.header("foo")
    got = cap._outputs[0]["data"]['text/markdown']

test_eq(got, '## foo')

In [28]:
with capture_output() as cap:
    streamlit.subheader("foo")
    got = cap._outputs[0]["data"]['text/markdown']

test_eq(got, '### foo')

In [ ]:
# these should fail

test_fail(lambda: streamlit.title(df), contains="Unsupported type")
test_fail(lambda: streamlit.header(df), contains="Unsupported type")
test_fail(lambda: streamlit.subheader(df), contains="Unsupported type")
test_fail(lambda: streamlit.subheader(1), contains="Unsupported type")

### patch some methods to simply display the input in jupyter 

In [24]:
# | exporti
def _st_type_check(
    func_to_decorate: tp.Callable, allowed_types: tp.Union[tp.Type , tp.Collection[tp.Type]]
) -> tp.Callable:
    """Decorator to display objects passed to Streamlit in Jupyter notebooks."""
    allowed_types = listify(allowed_types)  # make sure it's a list

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) == 1:
            body = args[0]
        elif len(args) > 1:
            raise ValueError(
                f"Too many positional arguments: {len(args)}, {func_to_decorate.__name__} only accepts 2"
            )
        elif len(args) == 0:
            if kwargs:
                raise NotImplementedError(
                    f"kwargs not supported yet, 'streamlit_data_science.utils._wrap_st_type_check' only accepts positional arguments"
                )
            else:
                raise ValueError(f"at least one positional argument is required")

        if type(body) in allowed_types:
            _display(body)
        else:
            raise TypeError(
                f"Unsupported type: {type(body)}, {func_to_decorate.__name__} only accepts {allowed_types}"
            )

    if IN_IPYTHON:
        return wrapper
    else:
        return func_to_decorate

wrapping 'streamlit.markdown' with 'streamlit_jupyter.core.functools.partial(<function _st_type_check at 0x16c1544c0>, allowed_types=<class 'str'>)'


In [31]:
sp._wrap('markdown', functools.partial(_st_type_check, allowed_types=str))

test_fail(lambda: streamlit.markdown(df), contains="Unsupported type")
streamlit.markdown("This is **bold** text in markdown")

wrapping 'streamlit.markdown' with 'streamlit_jupyter.core.functools.partial(<function _st_type_check at 0x16c1544c0>, allowed_types=<class 'str'>)'


This is **bold** text in markdown

In [ ]:
assert 0, 'stop here'

### StreamlitPatcher.MAPPING

Mapping is a dictionary that maps the streamlit method to the method we want to use instead.

This is used when StreamlitPatcher.jupyter() is called.

In [ ]:
# | exporti
@patch_to(StreamlitPatcher, as_prop=True)
def MAPPING(cls) -> tp.Dict[str, tp.Callable]:
    """mapping of streamlit methods to their jupyter friendly versions"""
    return {
        "write": _st_write,
        "title": functools.partial(_st_heading, tag="#"),
        'header': functools.partial(_st_heading, tag="##"),
        'subheader': functools.partial(_st_heading, tag="###"),  
        'markdown': functools.partial(_st_type_check, allowed_types=str),
        'dataframe': functools.partial(_st_type_check, allowed_types=pd.DataFrame),
         } 

In [32]:
streamlit.dataframe

<bound method DataFrameSelectorMixin.dataframe of DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)>

In [ ]:
sp = StreamlitPatcher()

In [ ]:
sp.MAPPING

{'write': <function __main__._st_write(func_to_decorate)>,
 'title': functools.partial(<function _st_heading at 0x1279cae50>, tag='#')}

## Calling tqdm as tqdm.notebook or stqdm depending on environment

In [ ]:
# | export
if IN_IPYTHON:
    from tqdm.notebook import tqdm
else:
    from stqdm import stqdm as tqdm

tqdm = tqdm  # make this available in the module namespace

In [ ]:
for i in tqdm(range(10)):
    pass

  0%|          | 0/10 [00:00<?, ?it/s]

# OLD CODE

## streamlitcache

The `streamlitcache` method is used to cache the output of a function. This is useful for functions that take a long time to run, and we want to avoid running them every time we run the app.

If we are in a jupyter notebook, we can't use the `streamlitcache` method, so we will replace the `streamlitcache` method with a dummy method that does nothing.

In [ ]:
def _dummy_wrapper_noop(func_to_decorate):
    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        return noop  # castrate the function to do nothing

    if IN_IPYTHON:
        return wrapper
    else:
        return func_to_decorate


streamlitcache = _dummy_wrapper_noop(streamlitcache)

In [ ]:
# verify that during patching we didn't change the name or docstring
assert streamlitcache.__name__ == "cache"
assert "@streamlitcache" in tp.cast(
    str, streamlitcache.__doc__
), "check that the docstring is correct"

In [ ]:
# test caching


@streamlitcache(suppress_st_warning=True)
def get_data():
    streamlit.write("Getting data...")
    for i in tqdm(range(5)):
        time.sleep(0.1)
    return pd.DataFrame({"c": [7, 8, 9], "d": [10, 11, 12]})


df = get_data()
streamlit.write(df)

Getting data...

  0%|          | 0/5 [00:00<?, ?it/s]

,c,d
0,7,10
1,8,11
2,9,12


In [ ]:
# test that the cache in jupyter does not affect get_data

df = get_data()
with capture_output() as cap:
    streamlit.write(df)
    got = cap._outputs[0]["data"]

expected = {
    "text/plain": "   c   d\n0  7  10\n1  8  11\n2  9  12",
    "text/html": '<div>\n<style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-align: middle;\n    }\n\n    .dataframe tbody tr th {\n        vertical-align: top;\n    }\n\n    .dataframe thead th {\n        text-align: right;\n    }\n</style>\n<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>c</th>\n      <th>d</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>7</td>\n      <td>10</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>8</td>\n      <td>11</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>9</td>\n      <td>12</td>\n    </tr>\n  </tbody>\n</table>\n</div>',
}

assert got == expected, "check that the output is correct"

Getting data...

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# patch streamlitexpander to display in jupyter


class _DummyExpander:
    def __init__(self, label: str, expanded: bool = False):
        self.label = label
        self.expanded = expanded

    def __enter__(self):
        _display(f">**expander starts**: {self.label}")

    def __exit__(self, *args):
        _display(f">**expander ends**")
        pass


if IN_IPYTHON:
    streamlitexpander = _DummyExpander  # type: ignore

In [ ]:
with streamlitexpander("Expand me!"):
    streamlitmarkdown(
        """
    The **#30DaysOfStreamlit** is a coding challenge designed to help you get started in building Streamlit apps.
    
    Particularly, you'll be able to:
    - Set up a coding environment for building Streamlit apps
    - Build your first Streamlit app
    - Learn about all the awesome input/output widgets to use for your Streamlit app
    """
    )

>**expander starts**: Expand me!


    The **#30DaysOfStreamlit** is a coding challenge designed to help you get started in building Streamlit apps.
    
    Particularly, you'll be able to:
    - Set up a coding environment for building Streamlit apps
    - Build your first Streamlit app
    - Learn about all the awesome input/output widgets to use for your Streamlit app
    

>**expander ends**

## Interactive Methods


In [ ]:
def _st_text_input(func_to_decorate):
    """Decorator to display objects passed to Streamlit in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) == 2:
            label, default = args
        elif len(args) == 1:
            label = args[0]
            default = kwargs["value"] if kwargs else input(f"{label}: ")
        else:
            raise ValueError(
                f"Too many positional arguments: {len(args)}, {func_to_decorate.__name__} only accepts 2"
            )

        streamlit.write(
            f"""```
############################################        
#       TEXT INPUT FIELD                   #
#                                          #
#   {label}: {default:<21} #
############################################        
"""
        )

    if IN_IPYTHON:
        return wrapper
    else:
        return func_to_decorate


streamlittext_input = _st_text_input(streamlittext_input)

In [ ]:
streamlittext_input("Enter some text", "default text")

```
############################################        
#       TEXT INPUT FIELD                   #
#                                          #
#   Enter some text: default text          #
############################################        


In [ ]:
streamlittext_input("Enter some text", value="foobar")

```
############################################        
#       TEXT INPUT FIELD                   #
#                                          #
#   Enter some text: foobar                #
############################################        


In [ ]:
# | notest
# streamlittext_input("Enter some text")

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()